![image](images/um_logo.png)

# Computación II: Concurrent Futures en Python

## Introducción a concurrent.futures

El módulo `concurrent.futures` es una parte poderosa de la biblioteca estándar de Python que proporciona una interfaz de alto nivel para trabajar con tareas concurrentes y paralelas. Este módulo es esencial para desarrolladores que buscan mejorar el rendimiento de sus aplicaciones aprovechando la concurrencia y el paralelismo.

### ¿Por qué usar concurrent.futures?

1. **Simplicidad**: Ofrece una API fácil de usar para la programación concurrente.
2. **Flexibilidad**: Permite cambiar fácilmente entre ejecución en hilos y procesos.
3. **Control**: Proporciona herramientas para manejar el estado y los resultados de las tareas.
4. **Escalabilidad**: Facilita la distribución de trabajo en múltiples núcleos de CPU.

## Motivación y Razón de Ser de concurrent.futures

El módulo `concurrent.futures` fue introducido en Python 3.2 como parte de la PEP 3148. Su principal objetivo es proporcionar una interfaz de alto nivel para la ejecución asíncrona de llamadas a funciones. Este módulo surge de la necesidad de simplificar y unificar la programación concurrente en Python, ofreciendo varias ventajas sobre el uso directo de `threading` y `multiprocessing`:

1. **Interfaz unificada**: `concurrent.futures` proporciona una API consistente para trabajar tanto con hilos (`ThreadPoolExecutor`) como con procesos (`ProcessPoolExecutor`). Esto permite cambiar fácilmente entre ejecución basada en hilos y procesos sin modificar significativamente el código.

2. **Abstracción de alto nivel**: A diferencia de `threading` y `multiprocessing`, que requieren una gestión manual de hilos y procesos, `concurrent.futures` abstrae estos detalles, permitiendo al programador centrarse en la lógica de la aplicación.

3. **Manejo simplificado de resultados**: El uso de objetos `Future` facilita la obtención de resultados de tareas asíncronas, proporcionando métodos para verificar el estado de las tareas y recuperar sus resultados.

4. **Pool de workers gestionado**: El módulo gestiona automáticamente un pool de workers (hilos o procesos), eliminando la necesidad de crear y gestionar manualmente estos recursos.

5. **Compatibilidad con contextos**: El uso de administradores de contexto (`with` statement) simplifica la gestión del ciclo de vida de los executors.

6. **Cancelación de tareas**: Ofrece mecanismos para cancelar tareas que aún no han comenzado a ejecutarse, una característica no fácilmente disponible en `threading` o `multiprocessing`.

Comparación con `threading` y `multiprocessing`:

- **Nivel de abstracción**: `concurrent.futures` opera a un nivel más alto, ocultando muchos de los detalles de implementación que `threading` y `multiprocessing` exponen.
- **Facilidad de uso**: Requiere menos código boilerplate (secciones de código que deben incluirse en muchos lugares con poca o ninguna alteración) y manejo manual de sincronización en comparación con `threading` y `multiprocessing`.
- **Flexibilidad**: Permite cambiar fácilmente entre ejecución basada en hilos y procesos, lo cual no es trivial cuando se usa directamente `threading` o `multiprocessing`.
- **Limitaciones**: Aunque es más fácil de usar, `concurrent.futures` puede ser menos flexible para escenarios muy específicos o complejos donde se necesita un control fino sobre hilos o procesos.

## Conceptos Clave

### 1. Executor
Es la base de `concurrent.futures`. Hay dos tipos principales:
- `ThreadPoolExecutor`: Para tareas limitadas por I/O.
- `ProcessPoolExecutor`: Para tareas limitadas por CPU.

### 2. Future
Representa el resultado pendiente de una operación asíncrona.

### 3. Pool
Un conjunto de workers (hilos o procesos) que ejecutan tareas.

### 4. Tareas CPU-bound vs I/O-bound

#### Tareas CPU-bound
Las tareas CPU-bound son aquellas que pasan la mayor parte del tiempo realizando cálculos en la CPU. Estas tareas están limitadas por la velocidad de procesamiento de la CPU.

Características de las tareas CPU-bound:
- Realizan cálculos intensivos
- No pasan mucho tiempo esperando por operaciones de entrada/salida
- Pueden beneficiarse significativamente del procesamiento paralelo en múltiples núcleos

Ejemplos de tareas CPU-bound:
- Cálculos matemáticos complejos
- Procesamiento de imágenes o video
- Simulaciones científicas
- Algoritmos de machine learning

Para tareas CPU-bound, `ProcessPoolExecutor` es generalmente más eficiente debido a que puede aprovechar múltiples núcleos de CPU.

#### Tareas I/O-bound
En contraste, las tareas I/O-bound pasan la mayor parte del tiempo esperando por operaciones de entrada/salida, como lecturas de disco, solicitudes de red, o interacciones con bases de datos.

Características de las tareas I/O-bound:
- Pasan mucho tiempo esperando por operaciones de E/S
- No realizan cálculos intensivos
- Pueden beneficiarse de la concurrencia, incluso en un solo núcleo

Ejemplos de tareas I/O-bound:
- Solicitudes a APIs web
- Operaciones de lectura/escritura en archivos
- Consultas a bases de datos
- Descarga de archivos

Para tareas I/O-bound, `ThreadPoolExecutor` es generalmente más eficiente debido a su menor overhead y capacidad para manejar múltiples operaciones de E/S concurrentemente.

## ThreadPoolExecutor vs ProcessPoolExecutor

| Característica | ThreadPoolExecutor | ProcessPoolExecutor |
|----------------|---------------------|----------------------|
| Uso ideal      | Tareas I/O bound    | Tareas CPU bound     |
| Overhead       | Bajo                | Alto                 |
| GIL            | Compartido          | Separado por proceso |
| Memoria        | Compartida          | Separada             |
| Comunicación   | Rápida              | Más lenta            |

## Ejemplos Prácticos

### 1. Uso básico de ThreadPoolExecutor

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

def worker(name):
    print(f"Worker {name} iniciando...")
    time.sleep(2)  # Simulamos una tarea que toma tiempo
    return f"Worker {name} terminó"

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(worker, f"A{i}") for i in range(5)]
    
    for future in futures:
        print(future.result())

print("Todas las tareas completadas")

### 2. Uso de ProcessPoolExecutor para tareas CPU-bound

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import math

def es_primo(n):
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

numeros = range(1, 100000, 1000)

with ProcessPoolExecutor() as executor:
    resultados = executor.map(es_primo, numeros)

primos = [num for num, es_primo in zip(numeros, resultados) if es_primo]
print(f"Números primos encontrados: {primos}")

### 3. Comparación de rendimiento

Vamos a comparar el rendimiento de ejecución secuencial, ThreadPoolExecutor y ProcessPoolExecutor para una tarea CPU-bound.

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def tarea_intensiva(n):
    return sum(i * i for i in range(n))

def ejecutar_secuencial(nums):
    return [tarea_intensiva(num) for num in nums]

def ejecutar_con_threads(nums):
    with ThreadPoolExecutor() as executor:
        return list(executor.map(tarea_intensiva, nums))

def ejecutar_con_procesos(nums):
    with ProcessPoolExecutor() as executor:
        return list(executor.map(tarea_intensiva, nums))

if __name__ == "__main__":
    numeros = [10**6, 10**6, 10**6, 10**6]  # Ajusta según tu CPU
    
    inicio = time.time()
    ejecutar_secuencial(numeros)
    tiempo_secuencial = time.time() - inicio
    
    inicio = time.time()
    ejecutar_con_threads(numeros)
    tiempo_threads = time.time() - inicio
    
    inicio = time.time()
    ejecutar_con_procesos(numeros)
    tiempo_procesos = time.time() - inicio
    
    print(f"Tiempo secuencial: {tiempo_secuencial:.2f} segundos")
    print(f"Tiempo con threads: {tiempo_threads:.2f} segundos")
    print(f"Tiempo con procesos: {tiempo_procesos:.2f} segundos")

### 4. Comparación rápida de código boilerplate

In [ ]:
# Con threading (más boilerplate)
import threading

def worker(num):
    print(f"Tarea {num} completada")

threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

# Con concurrent.futures (menos boilerplate)
from concurrent.futures import ThreadPoolExecutor

def worker(num):
    print(f"Tarea {num} completada")

with ThreadPoolExecutor() as executor:
    executor.map(worker, range(5))

## Ejercicios para Estudiantes

1. **Descarga de páginas web concurrente**: 
   Implementa un script que descargue el contenido de múltiples URLs utilizando `ThreadPoolExecutor`. Compara el tiempo de ejecución con una implementación secuencial.

2. **Procesamiento de imágenes paralelo**:
   Crea un programa que aplique un filtro (por ejemplo, escala de grises) a un conjunto de imágenes utilizando `ProcessPoolExecutor`. Compara el rendimiento con diferentes números de workers.

3. **Sistema de caché distribuido**:
   Diseña un sistema simple de caché distribuido utilizando `ProcessPoolExecutor`, donde cada proceso maneja una parte de los datos en memoria.

4. **Web scraping concurrente**:
   Desarrolla un web scraper que extraiga información de múltiples páginas de un sitio web simultáneamente usando `ThreadPoolExecutor`.

5. **Simulación de Monte Carlo paralela**:
   Implementa una simulación de Monte Carlo (por ejemplo, para calcular π) utilizando `ProcessPoolExecutor` y compara la precisión y el tiempo de ejecución con diferentes números de procesos.

## Referencias y Recursos Adicionales

- [Documentación oficial de concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html)
- [PEP 3148 -- futures - execute computations asynchronously](https://www.python.org/dev/peps/pep-3148/)

Recursos en línea gratuitos:
- [Python Threading Tutorial: Run Code Concurrently Using the Threading Module](https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python)
- [Parallel Processing in Python: A Practical Guide with Examples](https://www.machinelearningplus.com/python/parallel-processing-python/)
- [Python Multithreading and Multiprocessing Tutorial](https://www.tutorialspoint.com/python/python_multithreading.htm)